In [ ]:
from scipy import integrate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def getAdsorbedData(Bi = 72.1728,Thi = 0.00957,m = 1):

    Da = 0.000000000688 # m²/s
    E = 0.17 # adm
    Dp = Da*(E/(2-E))**2 # m²/s
    R = 0.000313 # m
    Ros = 630 # g/L
    b = 0.242176195290884 # L/mg
    qm = 19.8015873015873 # mg/g
    As = 93 # m²/g
    Vp = 0.4*10-6 # cm³/g
    V = 0.4 # L (dm³)
    Ds = 9.5703853591393e-13 # m²/s
    Ke = Bi*Dp/(2*R) # m/s
    Ki = Thi*E*Dp/(1000*As*Ros*R**2) # m/s
    C0 = 99.8 # mg/L


    # IVP: t = 0: q = c = 0

    def my_system(t, r):
        c, q = r
        C = C0 - (m/(Ros*V))*(c+Ros*q)
        dcdt = (15*Dp/(R**2))* ((Ke*R*C/5+Dp*c+Ros*Ds*q)\
                               /(Ke*R/5+Dp+Ros*Ds*b*qm/(1+b*c))-c) - (Ki*As*Ros/(E*Vp))*(b*qm*c/(1+b*c) - q)
        dqdt = (3*Ke/(R*Ros))*(C - (Ke*R*C/5+Dp*c+Ros*Ds*q)/(Ke*R/5+Dp+Ros*Ds*b*qm/(1+b*c))) - (dcdt/Ros)
        return dcdt, dqdt

    solution = integrate.solve_ivp(my_system, (0,2880*60), (0,0), method='RK45', t_eval=np.linspace(0,2880*60,2880*60+1))
    t = solution.t
    c, q = solution.y
    C = C0 - (m/(Ros*V))*(c+Ros*q)
    df_edo = pd.DataFrame()
    df_edo['tempo'] = t
    df_edo['tempo'] = df_edo['tempo']/60
    df_edo['c'] = c
    df_edo['q'] = q
    df_edo['C'] = C
    
    return df_edo

In [ ]:
df1 = getAdsorbedData(72.1728,0.00957,1)
df2 = getAdsorbedData(95.4114,0.0107,2)
df3 = getAdsorbedData(114.5648,0.0107,3)

In [ ]:
fig_edo = go.Figure()

fig_edo.add_trace(go.Scatter(x=df1['tempo'], y=df1['C'], name = '1/400', marker = dict(color = '#0000FF')))
fig_edo.add_trace(go.Scatter(x=df2['tempo'], y=df2['C'], name = '2/400', marker = dict(color = '#FF0000')))
fig_edo.add_trace(go.Scatter(x=df3['tempo'], y=df3['C'], name = '3/400', marker = dict(color = '#008000')))

fig_edo.update_layout(title='Concentração externa ao longo do tempo',
                   xaxis_title='Tempo (min)',
                   yaxis_title='Concentração (mg/L)',
                   width = 800,
                   height = 500)

fig_edo.show()

In [ ]:
df_exp = pd.read_csv("C:\\Users\\cgirao\\Desktop\\tcc_dados_experimentais.csv",sep=";")

In [ ]:
df_exp

In [ ]:
fig_edo.add_trace(go.Scatter(x=df_exp['t'], y=df_exp['C1'], name = '1/400 - exp', mode='markers', marker = dict(color = '#0000FF',symbol = 0)))
fig_edo.add_trace(go.Scatter(x=df_exp['t'], y=df_exp['C2'], name = '2/400 - exp', mode='markers', marker = dict(color = '#FF0000',symbol = 4)))
fig_edo.add_trace(go.Scatter(x=df_exp['t'], y=df_exp['C3'], name = '3/400 - exp', mode='markers', marker = dict(color = '#008000',symbol = 5)))
fig_edo.show()